In [7]:
# reviewer #2:
#. 3. The RNAseq dataset shown in this paper is underutilized and requires further analysis. 
# (A) The authors should show a ranking of the most predictive GO terms across both cell lines. 
# (B)They should also identify genes that are the best markers of proliferation from their dataset and show 
# how they compare to classical markers like PCNA and Ki67.

# show a ranking of the most predictive GO terms across both cell lines
import pandas as pd
import numpy as np
from xml.dom import minidom

# (1) load data
DATADIR = '/Volumes/CareyLab/Projects/2019__ProliferationCorrelatedExpression/Manuscript__ProlifCorrExpr/S Table/'
TABLEFILENAME = DATADIR + '/Table S4 GSEA results of Fibroblasts, ESCs and yeast sorted by CFSE or TMRE.xlsx'
df = pd.read_excel(TABLEFILENAME,skiprows=1)

# trailing whitespace is bad
df.columns = [x.strip() for x in df.columns]


df['average_NES'] = df.loc[:,['NES ESC','NES FIB']].mean(axis=1)
df = df.loc[ :, ['q.val' not in x for x in df.columns]]
df = df.loc[ :, ['SET SIZE' not in x for x in df.columns]]
df.sort_values('average_NES',inplace=True,ignore_index=True)

# round values to save space
df['NES ESC'] = np.round(df['NES ESC'],2)
df['NES FIB'] = np.round(df['NES FIB'],2)
df['NOM.p.val ESC'] = np.round(df['NOM.p.val ESC'],4)
df['NOM.p.val FIB'] = np.round(df['NOM.p.val FIB'],4)
df['average_NES'] = np.round(df['average_NES'],2)


# binary column if the NES has a different sign between the two cell types
df['same_sign'] =   ~np.logical_or(np.logical_and(df['NES ESC']>0,df['NES FIB']<0), np.logical_and(df['NES ESC']<0,df['NES FIB']>0))

df.head()

,GENE SET NAME,NES FIB,NOM.p.val FIB,NES ESC,NOM.p.val ESC,average_NES,same_sign
0,QUINTENS_EMBRYONIC_BRAIN_RESPONSE_TO_IR,-2.16,0.0,-2.19,0.0000,-2.17,True
1,MCMURRAY_TP53_HRAS_COOPERATION_RESPONSE_DN,-2.22,0.0,-1.95,0.0000,-2.08,True
2,FLORIO_NEOCORTEX_BASAL_RADIAL_GLIA_DN,-2.66,0.0,-1.26,0.0505,-1.96,True
3,FISCHER_G2_M_CELL_CYCLE,-2.40,0.0,-1.45,0.0000,-1.92,True
4,GO_NEGATIVE_REGULATION_OF_RETINOIC_ACID_RECEPT...,-1.81,0.0,-1.99,0.0000,-1.90,True


In [8]:
# (2) remove all non-predictive GO terms
df = df[ np.logical_or(df['NOM.p.val FIB']<0.05,df['NOM.p.val ESC']<0.05)]


In [11]:
# https://www.gsea-msigdb.org/gsea/downloads.jsp
#URL = 'https://www.gsea-msigdb.org/gsea/msigdb/download_file.jsp?filePath=/msigdb/release/7.2/msigdb_v7.2.xml'
#source = urllib.request.urlretrieve(URL)

#  load XML file with gene set descriptions
XML_FILE = '/Users/lcarey/Downloads/msigdb_v7.2.xml'
msigdb = minidom.parse(XML_FILE)

# add descriptions to the df
genesets = msigdb._get_childNodes()
genesets = genesets[1].childNodes

r = genesets[1]
r.attributes.items()

all_gene_sets_names = list()
all_gene_sets_desc_full  = list()
all_gene_sets_desc_brief  = list()

for gs in list(genesets):
    try: 
        len(gs) # this fails for Elements (what we want) 
    except:
        all_gene_sets_names.append( gs.attributes['STANDARD_NAME'].value )
        all_gene_sets_desc_full.append( gs.attributes['DESCRIPTION_FULL'].value )
        all_gene_sets_desc_brief.append( gs.attributes['DESCRIPTION_BRIEF'].value )  

genesets = pd.DataFrame({'GENE SET NAME':all_gene_sets_names, 
                        'desc':all_gene_sets_desc_brief,
                        'description_full':all_gene_sets_desc_full})
genesets.head()



,GENE SET NAME,desc,description_full
0,DAIRKEE_TERT_TARGETS_UP,Genes up-regulated in non-spontaneously immort...,An improved understanding of cell immortalizat...
1,KEEN_RESPONSE_TO_ROSIGLITAZONE_DN,Genes down-regulated in aorta biopsies from mi...,Diminished activity of peroxisome proliferator...
2,GRADE_COLON_CANCER_UP,Up-regulated genes in colon carcinoma tumors c...,To characterize patterns of global transcripti...
3,ROME_INSULIN_TARGETS_IN_MUSCLE_UP,Genes up-regulated by 3 h of euglycemic hyperi...,Insulin action in target tissues involved prec...
4,NIELSEN_SYNOVIAL_SARCOMA_DN,Top 20 negative significant genes associated w...,BACKGROUND: Soft-tissue tumours are derived fr...


In [12]:
# merge and save
df = df.merge(genesets,on='GENE SET NAME',how='left')
df.index.name = 'rank'
df.reset_index(inplace=True)
df.to_excel( DATADIR + 'ranking of the most predictive GO terms across both cell lines.xlsx'\
            ,index=False, freeze_panes=(1,1))
df.head()

,rank,GENE SET NAME,NES FIB,NOM.p.val FIB,NES ESC,NOM.p.val ESC,average_NES,same_sign,desc,description_full
0,0,QUINTENS_EMBRYONIC_BRAIN_RESPONSE_TO_IR,-2.16,0.0,-2.19,0.0000,-2.17,True,Genes up-regulated in the mouse embryonic brai...,The mammalian brain is especially sensitive to...
1,1,MCMURRAY_TP53_HRAS_COOPERATION_RESPONSE_DN,-2.22,0.0,-1.95,0.0000,-2.08,True,Down-regulated 'cooperation response genes': r...,Understanding the molecular underpinnings of c...
2,2,FLORIO_NEOCORTEX_BASAL_RADIAL_GLIA_DN,-2.66,0.0,-1.26,0.0505,-1.96,True,Genes down-regulated in basal radial glia (bRG...,Evolutionary expansion of the human neocortex ...
3,3,FISCHER_G2_M_CELL_CYCLE,-2.40,0.0,-1.45,0.0000,-1.92,True,Cell cycle genes with peak expression in G2/M ...,Cell cycle genes with a CC Expression Score >=...
4,4,GO_NEGATIVE_REGULATION_OF_RETINOIC_ACID_RECEPT...,-1.81,0.0,-1.99,0.0000,-1.90,True,"Any process that stops, prevents, or reduces t...",
